In [ ]:
import htcondor
import ipywidgets as widgets
from IPython.display import display, HTML
import os

max_form_width = '700px'
form_width = 'auto'
label_width = '160px'

def labeled_widget(widget, label, required=False):
    star = "<span style='color:red;'>*</span>" if required else ""
    return widgets.VBox([
        widgets.HTML(f"<label style='font-weight:600;font-family:Arial;'>{label} {star}</label>"),
        widget
    ])

script = widgets.Text(
    value='',
    placeholder='/home/you/project/my_job.sh',
    layout=widgets.Layout(width=form_width),
)

args = widgets.Text(
    value='',
    layout=widgets.Layout(width=form_width),
)

cpus = widgets.IntSlider(
    value=1, min=1, max=32,
    layout=widgets.Layout(width=form_width)
)

mem = widgets.IntText(
    value=1024,
    layout=widgets.Layout(width=form_width)
)

disk = widgets.IntText(
    value=1024,
    layout=widgets.Layout(width=form_width)
)

gpus = widgets.IntSlider(
    value=0, min=0, max=4,
    layout=widgets.Layout(width=form_width)
)

universe = widgets.Dropdown(
    options=['vanilla', 'docker', 'singularity'],
    value='vanilla',
    layout=widgets.Layout(width=form_width)
)

docker_image = widgets.Text(
    value='',
    placeholder='e.g. pytorch/pytorch:latest',
    layout=widgets.Layout(width=form_width)
)
docker_image.layout.display = 'none'

def on_uni_change(change):
    docker_image.layout.display = 'block' if change['new'] == 'docker' else 'none'
universe.observe(on_uni_change, names='value')

requirements = widgets.Text(
    value='',
    placeholder='e.g. OpSys=="LINUX"',
    layout=widgets.Layout(width=form_width)
)

transfer_in = widgets.Text(
    value='',
    placeholder='file1.txt,file2.dat',
    layout=widgets.Layout(width=form_width)
)

submit_btn = widgets.Button(
    description='Submit Job',
    button_style='success',
    layout=widgets.Layout(width='160px', height='40px'),
    tooltip='Submit the job to HTCondor'
)

preview_btn = widgets.Button(
    description='Preview Job',
    button_style='info',
    layout=widgets.Layout(width='160px', height='40px'),
    tooltip='Preview the submit description'
)

button_row = widgets.HBox(
    [submit_btn, preview_btn],
    layout=widgets.Layout(justify_content='center', gap='20px', margin='20px 0')
)

status_out = widgets.Output(layout=widgets.Layout(width='auto', min_height='4rem'))
status_box = widgets.VBox([status_out])
status_box.layout = widgets.Layout(
    width='100%',
    border='1px solid #ccc',
    padding='10px',
    border_radius='6px',
    background_color='#fafafa',
    min_height='6rem',
    overflow_y='auto',
    overflow_x='hidden',
)

form_grid = widgets.GridBox(
    children=[
        labeled_widget(script, "Script path:", required=True),
        labeled_widget(args, "Arguments:"),
        labeled_widget(cpus, "CPUs:"),
        labeled_widget(mem, "Memory (MB):"),
        labeled_widget(disk, "Disk (MB):"),
        labeled_widget(gpus, "GPUs:"),
        labeled_widget(universe, "Universe:"),
        labeled_widget(docker_image, "Docker image:"),
        labeled_widget(requirements, "Requirements:"),
        labeled_widget(transfer_in, "Transfer in:")
    ],
    layout=widgets.Layout(
        grid_template_columns='repeat(auto-fit, minmax(320px, 1fr))',
        grid_gap='20px 40px',
        width='100%'
    )
)

card = widgets.VBox(
    [
        widgets.HTML(
            "<h1 style='font-family: Arial, sans-serif; font-weight:700; text-align:center; margin-bottom:0;'>HTCondor Submission</h1>"
        ),
        widgets.HTML(
            "<p style='font-family: Arial, sans-serif; color:#666; text-align:center; margin-top:0; margin-bottom:20px;'>Fill in all required fields (marked with <span style='color:red;'>*</span>) and click Submit.</p>"
        ),
        form_grid,
        button_row,
        status_box,
    ],
    layout=widgets.Layout(
        width=max_form_width,
        padding='2rem',
        border='1px solid #ddd',
        border_radius='10px',
        box_shadow='0 6px 15px rgba(0, 0, 0, 0.1)',
        background_color='white',
        align_items='stretch',
        display='flex',
        flex_flow='column nowrap',
        margin='20px auto',
    )
)

container = widgets.Box(
    [card],
    layout=widgets.Layout(
        display='flex',
        align_items='center',
        justify_content='center',
        width='100%',
        min_height='100vh',
        padding='20px',
        background_color='#f0f2f5'
    )
)

def build_submit_dict():
    path = script.value.strip()
    if not path:
        raise ValueError("Script path is required.")
    if not os.path.isabs(path):
        path = os.path.abspath(path)
    submit_dict = {
        'executable': path,
        'arguments': args.value,
        'request_cpus': str(cpus.value),
        'request_memory': f"{mem.value}MB",
        'request_disk': f"{disk.value}MB",
        'request_gpus': str(gpus.value),
        'universe': universe.value,
        'output': '$(Cluster).$(Process).out',
        'error': '$(Cluster).$(Process).err',
        'log': '$(Cluster).log'
    }
    if universe.value == 'docker' and docker_image.value.strip():
        submit_dict['docker_image'] = docker_image.value.strip()
    if requirements.value.strip():
        submit_dict['requirements'] = requirements.value.strip()
    if transfer_in.value.strip():
        submit_dict['should_transfer_files'] = 'YES'
        submit_dict['transfer_input_files'] = transfer_in.value.strip()
    return submit_dict

def on_submit(b):
    b.disabled = True
    preview_btn.disabled = True
    status_out.clear_output()
    try:
        submit_dict = build_submit_dict()
        if not os.path.exists(submit_dict['executable']):
            raise FileNotFoundError(f"`{submit_dict['executable']}` not found.")
        schedd = htcondor.Schedd()
        desc = htcondor.Submit(submit_dict)
        with schedd.transaction() as txn:
            cid = desc.queue(txn)
        with status_out:
            print(f"Submitted as cluster {cid}")
    except Exception as e:
        with status_out:
            print(f"Failed: {e}")
    finally:
        b.disabled = False
        preview_btn.disabled = False

def on_preview(b):
    submit_btn.disabled = True
    b.disabled = True
    status_out.clear_output()
    try:
        submit_dict = build_submit_dict()
        submit = htcondor.Submit(submit_dict)
        with status_out:
            print("Preview of HTCondor Submit Description:")
            print("-" * 40)
            print(str(submit))
            print("-" * 40)
            print("This is not yet submitted.")
    except Exception as e:
        with status_out:
            print(f"Preview failed: {e}")
    finally:
        submit_btn.disabled = False
        b.disabled = False

submit_btn.on_click(on_submit)
preview_btn.on_click(on_preview)

status_box_manage = widgets.Output(layout=widgets.Layout(width='auto', max_height='400px', overflow_y='hidden', overflow_x='hidden'))

refresh_btn_manage = widgets.Button(
    description="Refresh Jobs",
    button_style='info',
    layout=widgets.Layout(width='160px', height='40px')
)

def job_status_label(status_code):
    status_map = {
        1: ("Idle", "#f0ad4e"),
        2: ("Running", "#5cb85c"),
        3: ("Removed", "#d9534f"),
        4: ("Completed", "#0275d8"),
        5: ("Held", "#f0ad4e"),
        6: ("Transferring Output", "#5bc0de"),
        7: ("Suspended", "#f7ec6e"),
    }
    return status_map.get(status_code, ("Unknown", "#999999"))

def refresh_jobs(b=None):
    status_box_manage.clear_output()
    with status_box_manage:
        try:
            schedd = htcondor.Schedd()
            jobs = schedd.query(
                projection=["ClusterId", "ProcId", "JobStatus", "Cmd", "Args", "EnteredCurrentStatus"],
                constraint=True
            )
            if not jobs:
                print("No jobs found.")
                return
            
            html = """
            <style>
                .job-table {
                    width: 100%;
                    border-collapse: collapse;
                    font-family: Arial, sans-serif;
                }
                .job-table th, .job-table td {
                    border: 1px solid #ddd;
                    padding: 8px 12px;
                    text-align: left;
                    vertical-align: middle;
                }
                .job-table th {
                    background-color: #007acc;
                    color: white;
                }
                .status-label {
                    padding: 3px 8px;
                    border-radius: 12px;
                    color: white;
                    font-weight: 600;
                    font-size: 0.85rem;
                    display: inline-block;
                    min-width: 80px;
                    text-align: center;
                }
                .job-row:hover {
                    background-color: #f1f9ff;
                }
            </style>
            <table class="job-table">
            <thead>
                <tr>
                    <th>Job ID</th>
                    <th>Status</th>
                    <th>Command</th>
                    <th>Arguments</th>
                </tr>
            </thead>
            <tbody>
            """
            for job in jobs:
                cid = job.get("ClusterId")
                pid = job.get("ProcId")
                status_code = job.get("JobStatus")
                status_text, color = job_status_label(status_code)
                cmd = job.get("Cmd", "")
                args = job.get("Args", "")
                job_id = f"{cid}.{pid}"
                html += f"""
                <tr class="job-row">
                    <td>{job_id}</td>
                    <td><span class="status-label" style="background-color: {color};">{status_text}</span></td>
                    <td>{cmd}</td>
                    <td>{args}</td>
                </tr>
                """
            html += "</tbody></table>"
            display(HTML(html))
        except Exception as e:
            print(f"Error fetching jobs: {e}")

refresh_btn_manage.on_click(refresh_jobs)

job_manage_page = widgets.VBox(
    [
        widgets.HTML(
            "<h2 style='margin-bottom:10px; font-family: Arial, sans-serif; font-weight:700; color:#007acc;'>HTCondor Job Management</h2>"
        ),
        widgets.HBox([refresh_btn_manage], layout=widgets.Layout(justify_content='flex-start', margin='0 0 10px 0')),
        status_box_manage
    ],
    layout=widgets.Layout(
        align_items='stretch',
        width=max_form_width,
        padding='1rem 1.5rem',
        border='1px solid #ddd',
        border_radius='10px',
        background_color='white',
        box_shadow='0 4px 12px rgba(0,0,0,0.05)'
    )
)

refresh_jobs()

tabs = widgets.Tab(children=[container, job_manage_page])
tabs.set_title(0, "Submit Job")
tabs.set_title(1, "Manage Jobs")

display(tabs)


